## Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from fastai.vision import *
from fastai.callbacks import *

## Seeing the Input and Dataset Files

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Unzipping Data

In [ ]:
!unzip /kaggle/input/aerial-cactus-identification/train.zip

In [ ]:
!unzip /kaggle/input/aerial-cactus-identification/test.zip

## Setting up DataBunch

In [ ]:
data = ImageDataBunch.from_csv(path = "/kaggle",
                               folder = "working/train/",
                               csv_labels = "input/aerial-cactus-identification/train.csv",
                               seed = 42,
                               test = "working/test/",
                               bs = 1024,
                               ds_tfms = get_transforms(),
                               size = 32,
                               num_workers = 6).normalize(imagenet_stats)
data

In [ ]:
print(data.classes)
print(data.c)

In [ ]:
data.show_batch(3, figsize=(6,6))

## Creating the Learner

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics = [accuracy, AUROC()])

## Fitting the Learner

In [ ]:
learn.fit_one_cycle(8)

In [ ]:
learn.save("model")

## Finetuning the Learner

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(16, max_lr = slice((1e-5)/2, 1e-4), callbacks = [SaveModelCallback(learn, name = "best_finetuned_model")])

## Interpreting the Learner

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

## Get Test Predictions

In [ ]:
test_preds, _ = learn.get_preds(DatasetType.Test)
print(len(test_preds))
test_preds

In [ ]:
test_preds[:,1]

## Creating DataFrame

In [ ]:
test_files = data.test_dl.items
test_files

In [ ]:
dic = {'id': test_files, 'has_cactus':test_preds[:,1]}
dic

In [ ]:
df = pd.DataFrame(dic)
df

In [ ]:
df['id'] = pd.Series(str(df['id'][i]).split("/")[-1] for i in range(df.shape[0]))
df

In [ ]:
df = df.sort_values(by=['id'], axis=0).reset_index(drop=True)
df

In [ ]:
df.to_csv("test_output.csv", index = False)